In [20]:
from bs4 import BeautifulSoup as bs
import datetime as dt
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')

In [21]:
cookies = {
    '__ddg1_': 'KWn3c1WwZzj6FysUsfDd',
    'ASPNET_SessionID': '5gqp1nhoznc4coucajtcdmt5',
    '_ym_uid': '172173650670206345',
    '_ym_d': '1721736506',
    '_ym_isad': '1',
    '_ym_visorc': 'b',
    'accept': '1',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    # 'cookie': '__ddg1_=KWn3c1WwZzj6FysUsfDd; ASPNET_SessionID=5gqp1nhoznc4coucajtcdmt5; _ym_uid=172173650670206345; _ym_d=1721736506; _ym_isad=1; _ym_visorc=b; accept=1',
    'priority': 'u=0, i',
    'referer': 'https://www.cbr.ru/currency_base/',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}

In [22]:
response = requests.get('https://www.cbr.ru/currency_base/daily/', cookies=cookies, headers=headers).text
soup = bs(response, 'html.parser')

list_header = []
data = []

In [23]:
header = soup.find_all("table")[0].find("tr")

for items in header:
    try: list_header.append(items.get_text())
    except: continue
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]

for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
    
df = pd.DataFrame(data=data, columns=list_header)

In [24]:
df = df.loc[df['Валюта'].str.contains('Доллар США|Евро')]

In [25]:
df['Курс'] = df['Курс'].str.replace(',', '.')
df['Курс'] = df['Курс'].astype(float) / df['Единиц'].astype(int)
df = df[['Букв. код', 'Курс']]
df.rename(columns={'Букв. код': 'Валюта'}, inplace=True)
df['Дата'] = dt.datetime.today().strftime('%d.%m.%Y')

In [93]:
response = requests.get('https://www.cbr.ru/hd_base/metall/metall_base_new/', cookies=cookies, headers=headers).text
soup = bs(response, 'html.parser')

list_header = []
data = []

In [94]:
header = soup.find_all("table")[0].find("tr")

for items in header:
    try: list_header.append(items.get_text())
    except: continue
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]

for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
    
df = pd.DataFrame(data=data, columns=list_header)

In [96]:
df.rename(columns={'Дата* ▼\n': 'Дата'}, inplace=True)

In [98]:
df = df[['Дата', 'Золото', 'Серебро', 'Платина', 'Палладий']][:1]
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

In [99]:
for name in df.columns[1:]:
    df[name] = df[name].str.replace(',', '.').str.replace(' ', '').astype(float)